Installation of required packages

In [2]:
! pip install psutil pandas pynvml tqdm line_profiler matplotlib


### Imports and initialization

In [3]:
import time
import tracemalloc
import psutil
import pandas as pd
import statistics
import logging
from functools import wraps

# Logging configuration: write execution details to a log file
logging.basicConfig(
    filename="benchmark.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

process = psutil.Process()

# Optional GPU memory tracking via NVIDIA NVML
GPU_AVAILABLE = False
try:
    import pynvml
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    GPU_AVAILABLE = True
except Exception:
    pass


### Single-run measurement utility

In [4]:
def run_once(fn, *args, **kwargs):
    """
    Run a function once and collect timing, CPU and memory statistics.
    Uses tracemalloc for peak Python-level memory and psutil for process memory.
    GPU usage recorded if NVML available.
    """
    cpu_before = process.cpu_percent(interval=None)
    mem_before = process.memory_info().rss

    tracemalloc.start()
    t0 = time.perf_counter()
    result = fn(*args, **kwargs)
    t1 = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    cpu_after = process.cpu_percent(interval=0.1)
    mem_after = process.memory_info().rss

    gpu_used = None
    if GPU_AVAILABLE:
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        gpu_used = info.used / 1e6

    return {
        "time_seconds": t1 - t0,
        "cpu_percent_avg": (cpu_before + cpu_after) / 2,
        "memory_used_MB": (mem_after - mem_before) / 1e6,
        "peak_memory_traced_MB": peak / 1e6,
        "gpu_memory_used_MB": gpu_used,
        "result": result,
    }


### Multi-run benchmark decorator

In [5]:
def benchmark(runs=5):
    """
    Decorator that executes a function multiple times and aggregates statistics.
    Returns mean and standard deviation for timing and resource usage.
    """
    def decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            records = []
            for _ in range(runs):
                rec = run_once(fn, *args, **kwargs)
                records.append(rec)

            return {
                "function": fn.__name__,
                "time_mean": statistics.mean(r["time_seconds"] for r in records),
                "time_std": statistics.stdev(r["time_seconds"] for r in records) if runs > 1 else 0,
                "cpu_mean": statistics.mean(r["cpu_percent_avg"] for r in records),
                "mem_mean_MB": statistics.mean(r["memory_used_MB"] for r in records),
                "peak_mem_mean_MB": statistics.mean(r["peak_memory_traced_MB"] for r in records),
                "gpu_mem_mean_MB": (
                    statistics.mean(r["gpu_memory_used_MB"] for r in records)
                    if GPU_AVAILABLE else None
                ),
            }
        return wrapper
    return decorator


### Example algorithms + parameters

In [6]:
@benchmark(runs=5)
def sort_list(n=500_000):
    """Reverse and sort a large list."""
    data = list(range(n))
    data.reverse()
    return sorted(data)

@benchmark(runs=5)
def fibonacci_iter(n=35):
    """Compute Fibonacci using an iterative loop (fast)."""
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a+b
    return a


### Test matrix

In [7]:
TESTS = [
    (sort_list, {"n": 200_000}),
    (sort_list, {"n": 1_000_000}),
    (fibonacci_iter, {"n": 35}),
    (fibonacci_iter, {"n": 45}),
]


### Sequential execution + progress bar

In [8]:
from tqdm import tqdm

results = []
for fn, params in tqdm(TESTS, desc="Sequential benchmarks"):
    res = fn(**params)
    logging.info(f"[SEQ] Bench {fn.__name__} params={params}: {res}")
    results.append({"params": params, **res})

df = pd.DataFrame(results)
df


Sequential benchmarks: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]


,params,function,time_mean,time_std,cpu_mean,mem_mean_MB,peak_mem_mean_MB,gpu_mem_mean_MB
0,{'n': 200000},sort_list,0.194722,0.053285,0.00,8.078950,9.621717,156.831744
1,{'n': 1000000},sort_list,0.853834,0.138639,10.08,39.800832,48.021569,156.831744
2,{'n': 35},fibonacci_iter,0.000049,0.000012,12.02,0.000000,0.000158,156.831744
3,{'n': 45},fibonacci_iter,0.000075,0.000024,0.00,0.000000,0.000159,156.831744


### Parallel execution (CPU multiprocessing)

In [ ]:
from multiprocessing import Pool, cpu_count

def run_single(param_tuple):
    """
    Wrapper to allow multiprocessing execution.
    """
    fn, params = param_tuple
    res = fn(**params)
    logging.info(f"[PARALLEL] Bench {fn.__name__} params={params}: {res}")
    return {"params": params, **res}

if __name__ == "__main__":
    with Pool(cpu_count()) as p:
        df_parallel = pd.DataFrame(p.map(run_single, TESTS))
df_parallel


In [ ]:
# Load the line profiler extension once at notebook start
%load_ext line_profiler

# Example profiling call (change function as needed)
%lprun -f sort_list sort_list(n=300_000)


In [ ]:
import matplotlib.pyplot as plt

def plot_metric(df, metric):
    """
    Produce simple exploratory visualizations for benchmarking metrics.
    Boxplot and violin plot are produced on separate figures.
    """
    plt.figure()
    df.boxplot(column=[metric])
    plt.title(f"Boxplot of {metric}")
    plt.show()

    plt.figure()
    df[metric].plot(kind='violin')
    plt.title(f"Violin plot of {metric}")
    plt.show()

# Generate plots for core metrics
for m in ["time_mean", "cpu_mean", "mem_mean_MB"]:
    plot_metric(df, m)


In [ ]:
df.to_csv("benchmark_full_results.csv", index=False)
df_parallel.to_csv("benchmark_parallel_results.csv", index=False)
print("All benchmarking complete.")
